In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import time
from scipy.ndimage import gaussian_filter
from skimage.morphology import binary_dilation, disk

# ***1. package definition***

In [2]:
import os
import numpy as np

import astra, tomopy
import tifffile, h5py
import matplotlib.pylab as mpp

def reader(fn, dtype, path=None, dim=None):
    if dtype == 'prj':
        with h5py.File(fn, 'r') as f:
            if dim is None:
                tem = -np.log((f['/img_tomo'][:] - f['/img_dark_avg'][:])/(f['/img_bkg_avg'][:] - f['/img_dark_avg'][:]))
                tem[np.isinf(tem)] = 1
                tem[np.isnan(tem)] = 1
            else:
                tem = -np.log((f['/img_tomo'][dim] - f['/img_dark_avg'][dim])/(f['/img_bkg_avg'][dim] - f['/img_dark_avg'][dim]))
                tem[np.isinf(tem)] = 1
                tem[np.isnan(tem)] = 1
        return tem
    if dtype == 'obj':
        if dim is None:
            return 0
        else:
            return np.zeros(dim)
    if dtype == 'theta':
        if dim is None:
            with h5py.File(fn, 'r') as f:
                if path is None:            
                    tem = f['/angle'][:]*np.pi/180
                else:
                    tem = f[path][:]*np.pi/180
            step = np.partition(np.diff(tem), 5)
            was = np.argmax(np.diff(tem))
            wan = int((tem[was+1] - tem[was])//step)
            return (np.concatenate(tem[:was], np.linspace(tem[was], tem[was+1], wan, endpoint=False)[:], tem[was+1:]),
                    was, was+wan-1)
        else:
            with h5py.File(fn, 'r') as f:
                if path is None:
                    tem = f['/angle'][:]*np.pi/180
                else:
                    tem = f[path][:]*np.pi/180
            return tem, dim[0], dim[1]
    if dtype == 'obj_supp':
        with h5py.File(fn, 'r') as f:
            if dim is None:
                if path is None:
                    return (f['/obj_supp'][:]>0).astype(np.int8)
                else:
                    return (f[path][:]>0).astype(np.int8)
            else:
                if path is None:
                    return (f['/obj_supp'][dim]>0).astype(np.int8)
                else:
                    return (f[path][dim]>0).astype(np.int8)
    if dtype == 'sin_supp':
        with h5py.File(fn, 'r') as f:
            if dim is None:
                if path is None:
                    return (f['/sin_supp'][:]>0).astype(np.int8)
                else:
                    return (f[path][:]>0).astype(np.int8)
            else:
                if path is None:
                    return (f['/sin_supp'][dim]>0).astype(np.int8)
                else:
                    return (f[path][dim]>0).astype(np.int8)
            
def bprjr(prj, cfg):
    """
    "back projector": prj -> obj
    Inputs:
        alg: str, algorithm name in ['gridrec', 'art', 'fbp', 'bart', 'sirt', 'tv',
                                     'mlem', 'osem', 'ospml_hybrid', 'ospml_quad', 
                                     'pml_hybrid', 'pml_quad', 'grad',
                                     'astra_3D', 'astra_2D']
        data: 'img' and 'theta' for alg in ['gridrec', 'art', 'fbp', 'bart', 'sirt', 'tv',
                                     'mlem', 'osem', 'ospml_hybrid', 'ospml_quad', 
                                     'pml_hybrid', 'pml_quad', 'grad']
              'img' and 'num_iters' for alg in ['astra_2D']
              'num_iters' for alg in ['astra_3D']
              
    """
    alg = cfg['name']
    if alg in ['gridrec', 'art', 'fbp', 'bart', 'sirt', 'tv', 'mlem', 'osem', 
               'ospml_hybrid', 'ospml_quad', 'pml_hybrid', 'pml_quad', 'grad']:
        # print(cfg['cfg'])
        mod_obj = tomopy.recon(prj, cfg['theta'], **cfg['cfg'])
    elif alg in ['astra.3D']:
        algorithm_id = astra.algorithm.create(cfg['cfg']['alg_cfg'])
        astra.algorithm.run(algorithm_id, cfg['cfg']['num_iters'])
        mod_obj = astra.data3d.get(cfg['cfg']['alg_cfg']['ReconstructionDataId'])
        astra.algorithm.delete(algorithm_id)
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ReconstructionDataId'])
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ProjectionDataId'])
        astra.functions.clear()
    elif alg in ['astra.2D']:
        mod_obj = []
        proj_geom = astra.data2d.get_geometry(cfg['cfg']['alg_cfg']['ProjectionDataId'])
        for ii in range(prj.shape[0]):
            cfg['cfg']['alg_cfg']['ProjectionDataId'] = astra.data2d.create('-sino', proj_geom, prj[ii])
            algorithm_id = astra.algorithm.create(cfg['cfg']['alg_cfg'])
            astra.algorithm.run(algorithm_id, cfg['cfg']['num_iters'])
            mod_obj.append(astra.data2d.get(cfg['cfg']['alg_cfg']['ReconstructionDataId']))
        astra.algorithm.delete(algorithm_id)
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ReconstructionDataId'])
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ProjectionDataId'])
        astra.functions.clear()
    return np.array(mod_obj)

def fprjr(obj, cfg):
    """
    "forward projector": obj -> prj
    inputs:
        obj: ndarray, 3D array of the object
        projector: str, type of projector in ['tomopy', 'astra']
        *args: positional arg theta for projector 'tomopy', and 'proj_geom', and
               'vol_geom' for projector 'astra'
    """
    if cfg['name'] == 'tomopy':
        proj = tomopy.project(obj, **cfg['cfg'])
    elif cfg['name'] == 'astra':
        fp_id, proj = astra.create_sino3d_gpu(obj, cfg['cfg'])
    return proj

def HIOEngine(inp):
    """
    The convergence is guarantted by the support constraint in obj space and 
    modulus constraint in the projection space. The support constraint in obj
    space uses HIO update scheme.
    Inputs:
        inp: dictionary
            inp['data']: 'prj', 'obj'
            inp['cnts']: 'use_pos_cnt', 'obj_supp'
            inp['prjr']: ['fwd']: 'name', 'cfg' (obj -> prj, 'theta' is defined in 'cfg')
                         ['bac']: 'name', 'cfg' (prj -> obj, 'ang_idx' is define in 'cfg', inp[])
            inp['HIO']: 'beta_obj'
    support constrain is set in slice space; magnitude constrain is set in sinogram space. in the terminology,
    sinogram space = proj space, slice space = obj space

    repeat:
        obj = tomopy.recon(mod_prj)
        prj = tomopy.project(obj)
        mod_prj = Prm(prj)
        mod_obj = tomopy.recon(mod_prj)
        Prm(obj)
            obj = mod_obj                     in support
            obj = obj - beta*mod_obj          not in support
    """
    # forward project (obj - prj), and apply modulus constraint in projection space
    mod_proj = fprjr(inp['data']['obj'], inp['prjr']['fwd'])
    # print(f"{inp['data']['obj'].shape = }, \n{inp['data']['prj'].shape = }, \n{mod_proj.shape = }", '\n',
    #       inp['data']['ang_idx'].shape, inp['prjr']['fwd']['cfg']['theta'].shape)
    mod_proj[inp['data']['ang_idx']] = inp['data']['prj'][inp['data']['ang_idx']]
    mod_proj *= inp['cnts']['sin_supp']

    if inp['cnts']['use_prj_pos_cnt']:
        mod_proj[mod_proj < 0] *= inp['cnts']['prj_pos_f']  
        
    # back project (prj -> obj), and apply support constraint in object space
    mod_obj = bprjr(mod_proj[inp['data']['ang_idx']], inp['prjr']['bac'])
    # print(f"{mod_obj.shape = }, \n{inp['cnts']['obj_supp'].shape = }")
    inp['data']['obj'][:] = (mod_obj * inp['cnts']['obj_supp'] + 
                             (inp['data']['obj'] - inp['HIO']['beta_obj'] * mod_obj) * 
                             (1 - inp['cnts']['obj_supp']))[:]

    
    if inp['cnts']['use_obj_pos_cnt']:
        inp['data']['obj'][inp['data']['obj']<0] *= inp['cnts']['obj_pos_f']
    
    
    """
    ### extra TV iteration -- start
    # forward project (obj - prj), and apply modulus constraint in projection space
    mod_proj = fprjr(inp['data']['obj'], inp['prjr']['fwd'])
    # print(f"{inp['data']['obj'].shape = }, \n{inp['data']['prj'].shape = }, \n{mod_proj.shape = }", '\n',
    #       inp['data']['ang_idx'].shape, inp['prjr']['fwd']['cfg']['theta'].shape)
    mod_proj[inp['data']['ang_idx']] = inp['data']['prj'][inp['data']['ang_idx']]

    if inp['cnts']['use_pos_cnt']:
        mod_proj[mod_proj < 0] *= 0.5  
        
    # back project (prj -> obj), and apply support constraint in object space
    tv_cfg = {'name':'tv', 'cfg':{}}
    tv_cfg['theta'] = cfg['inps']['prjr']['bac']['theta']
    tv_cfg['cfg']['algorithm'] = 'tv'
    tv_cfg['cfg']['center'] = cfg['inps']['prjr']['bac']['cfg']['center']
    tv_cfg['cfg']['num_gridx'] = cfg['inps']['prjr']['bac']['cfg']['num_gridx'] 
    tv_cfg['cfg']['num_gridy'] = cfg['inps']['prjr']['bac']['cfg']['num_gridx'] 
    tv_cfg['cfg']['reg_par'] = 0.3
    tv_cfg['cfg']['init_recon'] = inp['data']['obj']
    
    mod_obj = bprjr(mod_proj, tv_cfg)
    # print(f"{mod_obj.shape = }, \n{inp['cnts']['obj_supp'].shape = }")
    inp['data']['obj'][:] = (mod_obj * inp['cnts']['obj_supp'] + 
                             (inp['data']['obj'] - inp['HIO']['beta_obj'] * mod_obj) * 
                             (1 - inp['cnts']['obj_supp']))[:]
    if inp['cnts']['use_pos_cnt']:
        inp['data']['obj'][inp['data']['obj']<0] *= 0.5
    ### extra TV iteration -- end
    """

    return inp['data']['obj'], mod_proj, mod_obj

def wedge_recon(cfg):
    """
    This is a routine for reconstructing tomography dataset that has a missing
    angle range. The initial inputs of 'proj_modu' and 'theta', however, cover
    the entire 180 deg angle range. The data values in 'proj_modu' in the missing
    angle range are initialized to 0's. These values will be gradually updated 
    in the following iterations.
    
    Inputs:
        cfg: dictionary
            cfg['iter']: dictionary, iteration scheme
                        ['num_updates'], ['update_step'], ['num_HIO_iter']
            cfg['inps']: dictionary, inputs to HIOEngine
                        ['data']: ['prj'], ['obj'], 
                                  ['was'] (wedge angle start), 
                                  ['wae'] (wedge angle end)
                        ['cnts']: ['use_pos_cnt'], ['obj_supp']
                        ['prjr']: ['fwd']: ['name'], ['cfg'] (obj -> prj, 'theta' is defined in 'cfg')
                                  ['bac']: ['name'], ['cfg'] (prj -> obj, inp[])
                        ['HIO']: ['beta_obj']
            cfg['rcfg']: dictionary, debug setting
                        ['rec']: boolean, save intermediate results
                        ['path']: str, where to save intermediate results
    
    Returns:
    """
    if cfg['rcfg']['rec']:
        if not os.path.exists(cfg['rcfg']['path']):
            os.makedirs(cfg['rcfg']['path'], mode=0o777)
            
    cnt = 0
    for ii in range(cfg['iter']['num_updates']):
        p11_s = (cfg['inps']['data']['was'] + cnt*cfg['iter']['update_step'])
        p11_e = (cfg['inps']['data']['was'] + (cnt+1)*cfg['iter']['update_step'])
        # p12_s = (cfg['inps']['data']['was'] + (cnt-1)*cfg['iter']['update_step'])
        # p12_e = (cfg['inps']['data']['was'] + cnt*cfg['iter']['update_step'])
        
        p21_s = (cfg['inps']['data']['wae'] - (cnt+1)*cfg['iter']['update_step'] + 1)
        p21_e = (cfg['inps']['data']['wae'] - cnt*cfg['iter']['update_step'] + 1)
        # p22_s = (cfg['inps']['data']['wae'] - cnt*cfg['iter']['update_step'])
        # p22_e = (cfg['inps']['data']['wae'] - (cnt-1)*cfg['iter']['update_step'])
        print(f'working on angles with indices between {p11_s} and {p11_e}, and {p21_s} and {p21_e}')
                
        for jj in range(cfg['iter']['num_HIO_iter']):
            obj, proj, mod_obj = HIOEngine(cfg['inps'])
            cfg['inps']['prjr']['bac']['cfg']['init_recon'] = mod_obj
            print(f'\t update iteration: {ii}; HIO iteration: {jj}')
            #fn = os.path.join(cfg['rcfg']['path'], f"recon_obj_iter_{str(ii).zfill(3)}_HIO_{str(jj).zfill(3)}.tif")
            #tifffile.imsave(fn, obj.astype(np.float32))
            #fn = os.path.join(cfg['rcfg']['path'], f"recon_prj_iter_{str(ii).zfill(3)}_HIO_{str(jj).zfill(3)}.tif")
            #tifffile.imsave(fn, proj.astype(np.float32))

        """
        for jj in range(cfg['iter']['num_HIO_iter']):
            obj, proj, mod_obj = HIOEngine(cfg['tv_cfg'])
            print(f'\t update iteration: {ii}; TV iteration: {jj}')
        """
        
        print(f"{cfg['inps']['data']['prj'].shape = }, \n{proj.shape = }")
        cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(p11_e), np.arange(p21_s+1, cfg['inps']['data']['prj'].shape[0])))
        cfg['inps']['data']['prj'][p11_s:p11_e, :] = proj[p11_s:p11_e, :]
        cfg['inps']['data']['prj'][p21_s:p21_e, :] = proj[p21_s:p21_e, :]
        
        cfg['inps']['prjr']['bac']['theta'] = theta[cfg['inps']['data']['ang_idx']]

        if cfg['rcfg']['rec']:
            fn = os.path.join(cfg['rcfg']['path'], f"recon_obj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, obj.astype(np.float32))
            fn = os.path.join(cfg['rcfg']['path'], f"mod_obj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, mod_obj.astype(np.float32))
            #fn = os.path.join(cfg['rcfg']['path'], f"updated_sino_{str(ii).zfill(3)}.tif")
            #tifffile.imsave(fn, proj)
            fn = os.path.join(cfg['rcfg']['path'], f"updated_prj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, cfg['inps']['data']['prj'].astype(np.float32))
            fn = os.path.join(cfg['rcfg']['path'], f"raw_prj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, proj.astype(np.float32))
            
        if p11_e < p21_s:
            cnt += 1
        else:
            break
    return obj

# ***2. piece tests***

In [ ]:
prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]

prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415e.h5' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

In [ ]:
prj = reader(prj_fn, 'prj', dim=prj_dim)
obj = reader(obj_fn, 'obj', dim=obj_dim)
supp = reader(obj_supp_fn, 'obj_supp', dim=None)

In [ ]:
prj[was:wae+1, 0, :] = 0
plt.figure(0)
plt.imshow(prj[:, 0, :])

plt.figure(1)
plt.imshow(obj[0])

plt.figure(2)
plt.imshow(supp[:].astype(np.int8))

In [ ]:
plt.figure(0)
plt.imshow(cfg['inps']['cnts']['obj_supp'].astype(np.int8))
#print(cfg['inps']['data']['ang_idx'].shape)
#plt.plot(cfg['inps']['data']['ang_idx'])

In [ ]:
plt.figure(0)
plt.imshow(prj[:, 0, :])
plt.figure(1)
#plt.imshow(prj[:, 0, :]*(fprjr(cfg['inps']['cnts']['obj_supp'].astype(np.int8)[np.newaxis, :], cfg['inps']['prjr']['fwd'])[:, 0, :]>0))
plt.imshow(prj[:, 0, :]*(fprjr(supp[np.newaxis, :], cfg['inps']['prjr']['fwd'])[:, 0, :]>0))
plt.figure(2)
plt.imshow((fprjr(supp[np.newaxis, :], cfg['inps']['prjr']['fwd'])[:, 0, :]>0))

In [ ]:
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_sinogram_mask_590a.tif' 
tem = tifffile.imread(fn)
prj = np.ndarray([tem.shape[0], 1, tem.shape[1]])
prj[:, 0, :] = tem[:]
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_prj_mask_590a.tif' 
tifffile.imsave(fn, prj.astype(np.int8))

In [ ]:
cfg['inps']['prjr']['bac']['num_iter'] = 10
cfg['inps']['prjr']['bac']['theta'] = theta
cfg['inps']['prjr']['bac']['name'] = 'tv'
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = 'tv'
cfg['inps']['prjr']['bac']['cfg']['center'] = cen
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
cfg['inps']['prjr']['bac']['cfg']['reg_par'] = 0.1 
obj_supp = bprjr(prj, cfg['inps']['prjr']['bac'])
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_prj_mask_590a_iradon.tif' 
tifffile.imsave(fn, obj_supp)

In [ ]:
print(prj.shape, theta.shape)
plt.figure(0)
plt.imshow(obj_supp[0])
plt.figure(1)
plt.imshow(prj[:, 0, :])

# ***3. wedge recon***

In [ ]:
"""
prj_dim = np.s_[:, 1080:1081, 930:1520]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]
cen = 303.0
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66885_1080_mask_415e.h5' 
save_path = '/run/media/xiao/Data/data/Tiankai/66885_test_rec'
"""

prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]
cen = 307.5
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415e.h5' 
save_path = '/run/media/xiao/Data/data/Tiankai/66881_test_rec_mask_e'

######## initialization mlem round -- start
theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

# config
prj = reader(prj_fn, 'prj', dim=prj_dim)
prj[was:wae+1, 0, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

final_obj = tomopy.recon(prj[ang_idx], theta[ang_idx], algorithm='grad', center=cen, 
                       num_gridx=obj_dim[2], num_gridy=obj_dim[1], reg_par=0.1, num_iter=10)
######## initialization mlem round -- end

######## tv round -- start
theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
rec_save_fn = f'{save_path}/tv_round/final_rec.tiff'               

# config
cfg = {'iter':{}, 
       'inps':{'data':{}, 
               'cnts':{}, 
               'prjr':{'fwd':{'cfg':{}}, 
                       'bac':{'cfg':{}}}, 
               'HIO':{}}, 
       'tv_cfg':{'data':{}, 
                 'cnts':{}, 
                 'prjr':{'fwd':{'cfg':{}}, 
                         'bac':{'cfg':{}}}, 
                 'HIO':{}}, 
       'rcfg':{}}

cfg['iter']['num_updates'] = 1 #28
cfg['iter']['update_step'] = 51 #2
cfg['iter']['num_HIO_iter'] = 50

cfg['inps']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['inps']['data']['prj'][was:wae+1, 0, :] = 0
cfg['inps']['data']['obj'] = final_obj #reader(obj_fn, 'obj', dim=obj_dim)
cfg['inps']['data']['was'] = was
cfg['inps']['data']['wae'] = wae
cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['inps']['cnts']['use_obj_pos_cnt'] = False
cfg['inps']['cnts']['obj_pos_f'] = 0
cfg['inps']['cnts']['use_prj_pos_cnt'] = False
cfg['inps']['cnts']['prj_pos_f'] = 0.1
cfg['inps']['cnts']['obj_supp'] = reader(supp_fn, 'obj_supp', dim=None)
cfg['inps']['cnts']['sin_supp'] = reader(supp_fn, 'sin_supp', dim=None)[:, np.newaxis, :]

cfg['inps']['prjr']['fwd']['name'] = 'tomopy'
cfg['inps']['prjr']['fwd']['cfg']['theta'] = theta

cfg['inps']['prjr']['bac']['name'] = 'tv'
cfg['inps']['prjr']['bac']['theta'] = theta[cfg['inps']['data']['ang_idx']]
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = 'tv'
cfg['inps']['prjr']['bac']['cfg']['center'] = cen
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
cfg['inps']['prjr']['bac']['cfg']['reg_par'] = 0.1 
#cfg['inps']['prjr']['bac']['cfg']['init_recon'] = final_obj
cfg['inps']['HIO']['beta_obj'] = 0.7

cfg['rcfg']['rec'] = True
cfg['rcfg']['path'] = f'{save_path}/tv_round'

# recon
final_obj = wedge_recon(cfg)

tifffile.imsave(os.path.join(cfg['rcfg']['path'], 'final_rec.tiff'), 
                final_obj.astype(np.float32))
######## tv round -- end


######## mlem round -- start
theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
rec_save_fn = f'{save_path}/mlem/final_rec.tiff'               

# config
cfg = {'iter':{}, 
       'inps':{'data':{}, 
               'cnts':{}, 
               'prjr':{'fwd':{'cfg':{}}, 
                       'bac':{'cfg':{}}}, 
               'HIO':{}}, 
       'tv_cfg':{'data':{}, 
                 'cnts':{}, 
                 'prjr':{'fwd':{'cfg':{}}, 
                         'bac':{'cfg':{}}}, 
                 'HIO':{}}, 
       'rcfg':{}}

cfg['iter']['num_updates'] = 1 #28
cfg['iter']['update_step'] = 51 #2
cfg['iter']['num_HIO_iter'] = 50

cfg['inps']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['inps']['data']['prj'][was:wae+1, 0, :] = 0
cfg['inps']['data']['obj'] = final_obj #reader(obj_fn, 'obj', dim=obj_dim)
cfg['inps']['data']['was'] = was
cfg['inps']['data']['wae'] = wae
cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['inps']['cnts']['use_obj_pos_cnt'] = False
cfg['inps']['cnts']['obj_pos_f'] = 0.1
cfg['inps']['cnts']['use_prj_pos_cnt'] = False
cfg['inps']['cnts']['prj_pos_f'] = 0.1
cfg['inps']['cnts']['obj_supp'] = reader(supp_fn, 'obj_supp', dim=None)
cfg['inps']['cnts']['sin_supp'] = reader(supp_fn, 'sin_supp', dim=None)[:, np.newaxis, :]

cfg['inps']['prjr']['fwd']['name'] = 'tomopy'
cfg['inps']['prjr']['fwd']['cfg']['theta'] = theta

cfg['inps']['prjr']['bac']['name'] = 'mlem'
cfg['inps']['prjr']['bac']['theta'] = theta[cfg['inps']['data']['ang_idx']]
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = cfg['inps']['prjr']['bac']['name']
cfg['inps']['prjr']['bac']['cfg']['center'] = cen
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
#cfg['inps']['prjr']['bac']['cfg']['init_recon'] = final_obj

cfg['inps']['HIO']['beta_obj'] = 0.7

cfg['rcfg']['rec'] = True
cfg['rcfg']['path'] = f'{save_path}/mlem'

# recon
final_obj = wedge_recon(cfg)

tifffile.imsave(os.path.join(cfg['rcfg']['path'], 'final_rec.tiff'), 
                final_obj.astype(np.float32))
######## mlem round -- end


# imshow
mpp.imshow(cfg['inps']['data']['obj'][0, :])


mpp.title('rec_wedge_sirt_no_patch')
mpp.show()


In [ ]:
prj_dim = np.s_[:, 1080:1081, 930:1520]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]

prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66885_1080_mask_415d.h5' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/final_rec.tiff'               

# config
cfg = {'iter':{}, 
       'inps':{'data':{}, 
               'cnts':{}, 
               'prjr':{'fwd':{'cfg':{}}, 
                       'bac':{'cfg':{}}}, 
               'HIO':{}}, 
       'tv_cfg':{'data':{}, 
                 'cnts':{}, 
                 'prjr':{'fwd':{'cfg':{}}, 
                         'bac':{'cfg':{}}}, 
                 'HIO':{}}, 
       'rcfg':{}}

cfg['iter']['num_updates'] = 28
cfg['iter']['update_step'] = 2
cfg['iter']['num_HIO_iter'] = 5

cfg['inps']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['inps']['data']['prj'][was:wae+1, 0, :] = 0
cfg['inps']['data']['obj'] = reader(obj_fn, 'obj', dim=obj_dim)
cfg['inps']['data']['was'] = was
cfg['inps']['data']['wae'] = wae
cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['inps']['cnts']['use_pos_cnt'] = False
cfg['inps']['cnts']['obj_supp'] = reader(obj_supp_fn, 'obj_supp', dim=None)

cfg['inps']['prjr']['fwd']['name'] = 'tomopy'
cfg['inps']['prjr']['fwd']['cfg']['theta'] = theta

cfg['inps']['prjr']['bac']['name'] = 'mlem'
cfg['inps']['prjr']['bac']['theta'] = theta
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = cfg['inps']['prjr']['bac']['name']
cfg['inps']['prjr']['bac']['cfg']['center'] = 303
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
# cfg['inps']['prjr']['bac']['cfg']['reg_par'] = 0.1 

cfg['inps']['HIO']['beta_obj'] = 0.7

cfg['tv_cfg']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['tv_cfg']['data']['prj'][was:wae+1, 0, :] = 0
cfg['tv_cfg']['data']['obj'] = reader(obj_fn, 'obj', dim=obj_dim)
cfg['tv_cfg']['data']['was'] = was
cfg['tv_cfg']['data']['wae'] = wae
cfg['tv_cfg']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['tv_cfg']['cnts']['use_pos_cnt'] = False
cfg['tv_cfg']['cnts']['obj_supp'] = reader(obj_supp_fn, 'obj_supp', dim=None)

cfg['tv_cfg']['prjr']['fwd']['name'] = 'tomopy'
cfg['tv_cfg']['prjr']['fwd']['cfg']['theta'] = theta

cfg['tv_cfg']['prjr']['bac']['name'] = 'tv'
cfg['tv_cfg']['prjr']['bac']['theta'] = theta
cfg['tv_cfg']['prjr']['bac']['cfg']['algorithm'] = 'tv'
cfg['tv_cfg']['prjr']['bac']['cfg']['center'] = 303
cfg['tv_cfg']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['tv_cfg']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
cfg['tv_cfg']['prjr']['bac']['cfg']['reg_par'] = 0.1 
cfg['tv_cfg']['HIO']['beta_obj'] = 0.7

cfg['rcfg']['rec'] = True
cfg['rcfg']['path'] = '/run/media/xiao/Data/data/Tiankai/test_rec'

# recon
cfg['inps']['data']['obj'] = wedge_recon(cfg)

tifffile.imsave(os.path.join(cfg['rcfg']['path'], 'final_rec.tiff'), 
                cfg['inps']['data']['obj'].astype(np.float32))

# imshow
mpp.imshow(cfg['inps']['data']['obj'][0, :])
mpp.title('rec_wedge_sirt_no_patch')
mpp.show()


# ***4. naive tomopy recon***

In [ ]:
prj_dim = np.s_[:, 1080:1081, 930:1520]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]

######## mlem round
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66885_1080_mask_415d.h5' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/mlem/final_rec.tiff'               

# config
prj = reader(prj_fn, 'prj', dim=prj_dim)
prj[was:wae+1, 0, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

mod_obj = tomopy.recon(prj[ang_idx], theta[ang_idx], algorithm='grad', center=303, 
                       num_gridx=obj_dim[2], num_gridy=obj_dim[1], reg_par=0.1, num_iter=50)

fp_cfg = {'name':'tomopy', 'cfg':{'theta':theta}}
mod_prj = fprjr(mod_obj, fp_cfg)

In [ ]:
plt.figure(0)
plt.imshow(mod_obj[0], vmin=0, vmax=7e-3)

plt.figure(1)
plt.imshow(mod_prj[:, 0, :])

plt.figure(2)
plt.imshow(mod_prj[ang_idx, 0, :])

plt.figure(3)
plt.imshow(prj[ang_idx, 0, :])

In [ ]:
plt.plot(theta)

# ***naive astra recon***

In [3]:
prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
ts = 122
theta_dim = [ts, ts+100]
#theta_dim = [122, 222]
cen = 307.5 - (prj_dim[2].stop - prj_dim[2].start)/2
print(f'{cen = }')
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415p.h5' 
rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/final_rec.tiff' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
#sin_supp = gaussian_filter(reader(supp_fn, 'sin_supp'), 3)
sin_supp = gaussian_filter(reader(supp_fn, 'sin_supp').astype(np.float32), 4)

#rec_supp = gaussian_filter(binary_dilation(reader(supp_fn, 'obj_supp'), disk(3)).astype(np.float32), 3)
rec_supp = binary_dilation(reader(supp_fn, 'obj_supp'), disk(3))
#rec_supp = gaussian_filter(reader(supp_fn, 'obj_supp').astype(np.float32), 5)
#rec_supp = reader(supp_fn, 'obj_supp')


prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
prj -= ((prj.sum(axis=1)-220)/prj.shape[1])[:, np.newaxis]
prj[was:wae+1, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

#prj = tifffile.imread('/run/media/xiao/Data/data/Tiankai/66881_1080_sinogram_590a_mask.tif')
print(f"{prj.shape = }, {prj[ang_idx].shape = }, {theta[ang_idx].shape = }")

cen = 12.5
prj.shape = (391, 590), prj[ang_idx].shape = (290, 590), theta[ang_idx].shape = (290,)


<ipython-input-2-82e1b16022bd>:16: RuntimeWarning: divide by zero encountered in log
  tem = -np.log((f['/img_tomo'][dim] - f['/img_dark_avg'][dim])/(f['/img_bkg_avg'][dim] - f['/img_dark_avg'][dim]))
<ipython-input-2-82e1b16022bd>:16: RuntimeWarning: invalid value encountered in log
  tem = -np.log((f['/img_tomo'][dim] - f['/img_dark_avg'][dim])/(f['/img_bkg_avg'][dim] - f['/img_dark_avg'][dim]))


In [27]:
print(f"    recon starts at {time.asctime()}")
proj_geom = astra.create_proj_geom('parallel', 1., 590, theta[ang_idx])
#proj_geom = astra.create_proj_geom('parallel', 1., 590, theta)
vol_geom = astra.create_vol_geom(415, 415)

recon_id = astra.data2d.create('-vol', vol_geom, 0)
rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
proj_id = astra.data2d.create('-sino', proj_geom, prj[ang_idx])
sin_supp_id = astra.data2d.create('-sino', proj_geom, sin_supp[ang_idx])
#proj_id = astra.data2d.create('-sino', proj_geom, prj)
#sin_supp_id = astra.data2d.create('-sino', proj_geom, sin_supp)

#proj_geom_vec = astra.functions.geom_2vec(proj_geom)
#proj_geom_vec['Vectors'][:, 11] *= np.sin(np.pi/2-alpha)
#proj_id = astra.data3d.create('-sino', proj_geom_vec, imgs)

astra.functions.geom_postalignment(proj_geom, cen)

alg_cfg = astra.astra_dict('SIRT_CUDA')
#alg_cfg = astra.astra_dict('SIRT3D_CUDA')
#alg_cfg = astra.astra_dict('BP3D_CUDA')

alg_cfg['ProjectionDataId'] = proj_id #sinogram_id #
alg_cfg['ReconstructionDataId'] = recon_id
alg_cfg['option'] = {}
#alg_cfg['option']['SinogramMaskId'] = sin_supp_id
alg_cfg['option']['ReconstructionMaskId'] = rec_supp_id
alg_cfg['option']['MaxConstraint'] = 2e-2
alg_cfg['option']['MinConstraint'] = 1e-4

algorithm_id = astra.algorithm.create(alg_cfg)
astra.algorithm.run(algorithm_id, 10000)
recon = astra.data2d.get(recon_id)
print(f"    recon finishes at {time.asctime()}")

#astra_mex_data2d('change_geometry', id, geom)
fp_geom = astra.create_proj_geom('parallel', 1., 590, theta)
fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino(recon, fp_id)
print(f"    forward projection finishes at {time.asctime()}")

astra.algorithm.delete(algorithm_id)
astra.data2d.delete(recon_id)
astra.data2d.delete(proj_id)
astra.data2d.delete(sinogram_id)
astra.projector.delete(fp_id)
astra.functions.clear()

    recon starts at Sat Dec  5 09:45:35 2020
    recon finishes at Sat Dec  5 09:45:46 2020
    forward projection finishes at Sat Dec  5 09:45:46 2020


In [28]:
plt.figure(10)
plt.imshow(recon, vmin=-1e-4, vmax=10e-3)
#plt.imshow(rec_supp)

plt.figure(11)
plt.imshow(sinogram, vmin=0, vmax=3)

plt.figure(12)
plt.imshow(prj, vmin=0, vmax=3)

plt.figure(13)
#plt.imshow(prj*(sinogram!=0))
plt.imshow(rec_supp)

plt.figure(14)
plt.imshow(prj-sinogram, vmin=0, vmax=3)

In [19]:
plt.figure(5)
plt.plot(sinogram.sum(axis=1))
#plt.plot(sinogram.sum(axis=1))

In [ ]:
prj = tifffile.imread('/run/media/xiao/Data/data/Tiankai/66881_1080_sinogram_590a_mask.tif')

In [ ]:
tifffile.imsave('/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415p_sinogram.tif', (sinogram>0).astype(np.int8))
#tifffile.imsave('/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415o.tif', recon.astype(np.float32))
#tifffile.imsave('/run/media/xiao/Data/data/Tiankai/66881_1080_sinogram_mask_590f.tif', (prj-sinogram).astype(np.float32))

In [ ]:
import scipy.io
P = scipy.io.loadmat('/home/xiao/Downloads/phantom.mat')['phantom256']
plt.imshow(P)


In [ ]:
help(gaussian_filter)
#dir(astra)
#help(astra.create_sino)

In [ ]:
print(rec_supp)
print(reader(supp_fn, 'obj_supp'))

In [ ]:
plt.figure(20)
plt.imshow(sin_supp)